In [1]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
import tensorflow as tf
from PIL import Image
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from numpy.random import seed

2023-07-17 14:06:50.117733: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-17 14:06:51.659087: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-17 14:07:02.201699: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
seed(0)
tf.random.set_seed(0)

In [3]:
#DEFINE AND AUGMENT TRAIN DATA
train_ds = tf.keras.utils.image_dataset_from_directory(
    './Datasheet_RPS/train',
    labels='inferred',
    label_mode='int',
    class_names=None,
    batch_size=32,
    image_size=(30, 30),
    shuffle=True,
    seed=10,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)
print (train_ds)

Found 2530 files belonging to 3 classes.
<_BatchDataset element_spec=(TensorSpec(shape=(None, 30, 30, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [4]:
#DEFINE AND AUGMENT VALIDATION DATA
val_ds = tf.keras.utils.image_dataset_from_directory(
    './Datasheet_RPS/valid',
    labels='inferred',
    label_mode='int',
    class_names=None,
    batch_size=32,
    image_size=(30, 30),
    shuffle=True,
    seed=10,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)
print(train_ds)

Found 372 files belonging to 3 classes.
<_BatchDataset element_spec=(TensorSpec(shape=(None, 30, 30, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [5]:

#NORMALIZATION
normalization_layer = layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))


2023-07-17 14:07:26.269829: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2530]
	 [[{{node Placeholder/_4}}]]
2023-07-17 14:07:26.270689: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2530]
	 [[{{node Placeholder/_0}}]]


In [6]:

#CREATE MODEL
model = keras.Sequential()
model.add(layers.Conv2D(90, 3, activation='relu', input_shape=(30,30,3)))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(64, 3, activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(32, 3, activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3))


In [7]:

from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning, PruningSummaries

pruning_params = {

    "pruning_schedule": pruning_schedule.PolynomialDecay(
    initial_sparsity = 0,
    final_sparsity = .9,
    begin_step = 0,
    end_step = 3000,
    power=3,
    frequency=100)
    

    }
model = prune.prune_low_magnitude(model, **pruning_params)


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d   (None, 28, 28, 90)       4952      
 (PruneLowMagnitude)                                             
                                                                 
 prune_low_magnitude_max_poo  (None, 14, 14, 90)       1         
 ling2d (PruneLowMagnitude)                                      
                                                                 
 prune_low_magnitude_conv2d_  (None, 12, 12, 64)       103746    
 1 (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_max_poo  (None, 6, 6, 64)         1         
 ling2d_1 (PruneLowMagnitude                                     
 )                                                               
                                                        

In [9]:
optimizer=tf.keras.optimizers.Adam()
optimizer.learning_rate.assign(0.00008)

call = []
call.append(pruning_callbacks.UpdatePruningStep())
call.append(tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.001,
    patience=10,
    verbose=0,
    mode="min",
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=38,
))
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy']
             )

In [ ]:
epochs=150

history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  callbacks=call
)
model = strip_pruning(model)

Epoch 1/150


2023-07-17 14:07:28.570623: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2530]
	 [[{{node Placeholder/_0}}]]
2023-07-17 14:07:28.571132: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2530]
	 [[{{node Placeholder/_0}}]]


In [ ]:
w = model.layers[0].weights[0].numpy()
h, b = np.histogram(w, bins=100)
plt.figure(figsize=(7, 7))
plt.bar(b[:-1], h, width=b[1] - b[0])
plt.semilogy()
print('% of zeros = {}'.format(np.sum(w == 0) / np.size(w)))

In [ ]:
model.save("my_model.keras")

In [ ]:
epochs=epochs
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()